In [1]:
from pyspark import SparkContext
import pyspark
sc = SparkContext()

In [2]:
data = sc.textFile("train.csv").map(lambda line: line.split(",")).filter(lambda line: len(line)>1).map(lambda line: (line[0]+ "$%$" +line[5])) 

In [3]:
import gensim
import re
import nltk
nltk.download("punkt")
import pyspark
import numpy as np
from string import punctuation
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()
from sklearn.model_selection import train_test_split
from collections import Counter
from operator import add
from pyspark.mllib.evaluation import BinaryClassificationMetrics

[nltk_data] Downloading package punkt to /home/akki/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# !pip install gensim

In [5]:
data.collect()

reduced_data_1 = data.filter(lambda x: x[0]=='4')
reduced_data_0 = data.filter(lambda x: x[0]=='0')
print(reduced_data_1.count())
print(reduced_data_0.count())
# zero_data = reduced_data_0.takeSample(False,248576,seed=None)
# one_data = reduced_data_1.takeSample(False,248576,seed=None)
zero_data = reduced_data_0.takeSample(False,50,seed=None)
one_data = reduced_data_1.takeSample(False,50,seed=None)

all_data = sc.parallelize(zero_data).union(sc.parallelize(one_data))

labels = all_data.map(lambda x: x.split("$%$")[0])
print(labels.count())
x = labels.map(lambda x: (x,1)).reduceByKey(add)
x.collect()

248576
800000
100


[('4', 50), ('0', 50)]

In [6]:
def tokenize_(tweet):
    tweet_low2 = str(tweet).lower()
    tweet_low3 = tweet_low2.replace(r"\n","")
    punctuations = '''!()-[]{};:'"\,<>./?$%^&*_~'''
    tweet = ""
    tweet_low = tweet_low3
    if(tweet_low[0:4]=="b'rt"):
        filtered_tweet = tweet_low[4:]
    else:
        filtered_tweet = tweet_low[2:]
    for e in filtered_tweet:
        if e not in punctuations:
            tweet+=e
    tweet_list = tweet.split(" ")
    tweet = ""
    for e in tweet_list:
        if(e!=""):
            if e[0]!='@' and e[0]!='#' and e[0:5]!="https" and e[0:4]!="http":
                if(tweet!=""):
                    tweet += e + " "
                else:
                    tweet = e + " "
    tokens = nltk.word_tokenize(tweet)
    return tokens

data_top_100_rdd = all_data
texts = data_top_100_rdd.map(lambda x: x.split("$%$")[1])
# labels = data_top_100_rdd.map(lambda x: x.split("$%$")[0])

clean_texts=texts.map(lambda x: tokenize_(x))
all_tweets = np.array(clean_texts.collect())

In [7]:
from gensim.models import FastText
model = FastText([x for x in all_tweets], min_count=1,size=10)

In [8]:
from gensim.models import FastText
def buildWordVector(tokens):
    size=10
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        vec += model.wv[word].reshape((1, size))
        count += 1.
    if count != 0:
        vec /= count
    return vec

all_tweets_ft = clean_texts.map(lambda x :buildWordVector(x))

In [9]:
def change_label(x):
    if(int(x)==4):
        return 1
    return 0
    
def join_labels(x,all_labels,all_data_2):
    index = x[1]
    data_point = np.concatenate((all_labels[index],np.array(all_data_2[index])),axis=1)
    return data_point

labels2 = labels.map(change_label)
all_labels = [[[l]] for l in np.array(labels2.collect())]
all_data_2 = np.array(all_tweets_ft.collect())
index_labels = labels2.zipWithIndex()

all_data_labeled = index_labels.map(lambda x : join_labels(x,all_labels,all_data_2))

In [10]:


# all_data_labeled.first()

tr,te = all_data_labeled.randomSplit([0.8, 0.2])
train = tr.map(lambda x:x[0][1:])
test = te.map(lambda x:x[0][1:])
train_labels = np.array(tr.map(lambda x: x[0][0]).collect())
test_labels = np.array(te.map(lambda x: x[0][0]).collect())



In [11]:
from pyspark.mllib.regression import LabeledPoint
def parsing(x):
    y = np.array(x[0])
    return LabeledPoint(int(y[0]), y[1:])

parse_tr = tr.map(parsing)
parse_te = te.map(parsing)
# parse.collect()

In [12]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
dtmodel = DecisionTree.trainClassifier(parse_tr, numClasses=2, categoricalFeaturesInfo={},impurity='gini', maxDepth=5, maxBins=32)

In [13]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel
svmmodel = SVMWithSGD.train(parse_tr, iterations=1000)

In [14]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
lrmodel = LogisticRegressionWithLBFGS.train(parse_tr)

In [15]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
rfmodel = RandomForest.trainClassifier(parse_tr, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=4, featureSubsetStrategy="auto" ,impurity='gini', maxDepth=4, maxBins=32)


In [16]:
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
gbmodel = GradientBoostedTrees.trainClassifier(parse_tr,
                                             categoricalFeaturesInfo={}, numIterations=10)

In [17]:
predictions = []
    
predictions.append(np.array(svmmodel.predict(test).collect()))
predictions.append(np.array(gbmodel.predict(test).collect()))
predictions.append(np.array(rfmodel.predict(test).collect()))
predictions.append(np.array(lrmodel.predict(test).collect()))
predictions.append(np.array(dtmodel.predict(test).collect()))

print(predictions)

[array([1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1]), array([1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0.]), array([1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0.]), array([1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1]), array([0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.])]


In [18]:
predictions_np = np.array(predictions)
cols = []
for i in range(len(predictions_np[0])):
    cols.append(predictions_np[:,i])

In [19]:
def sum_(x):
    return np.sum(x)
count_ones = sc.parallelize(cols).map(lambda x: sum_(x))

print(count_ones)
def give_label(x):
    if(int(x)>2):
        return 1
    return 0
final_predictions = count_ones.map(lambda x: give_label(x))
final_predictions.collect()

PythonRDD[435] at RDD at PythonRDD.scala:53


[1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0]

In [20]:

tp=0
fp=0
tn=0
fn=0
acc=0
dd = np.array(final_predictions.collect())
for i in range(len(dd)):
    if(dd[i]==test_labels[i]):
        acc+=1
        if(dd[i]==0):
            tp+=1
        else:
            tn+=1
    else:
        if(dd[i]==0):
            fn+=1
        else:
            fp+=1
print(acc*100.0/len(dd))
print("")
print(tp/(tp+fn))
print(tn/(tn+fp))
print(fp/(fp+tn))
print(fn/(fn+tp))



50.0

0.5
0.5
0.5
0.5
